In [ ]:
!pip install --upgrade transformers
!pip install --upgrade datasets
!pip install --upgrade huggingface_hub
!pip install evaluate
!pip install accelerate
!pip install ptflops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
from datasets import load_dataset
from huggingface_hub import login
from transformers import AutoImageProcessor, AutoFeatureExtractor
from torchvision.transforms import (CenterCrop,
                                    Compose,
                                    Normalize,
                                    RandomHorizontalFlip,
                                    RandomResizedCrop,
                                    Resize,
                                    ToTensor)
from torch.utils.data import DataLoader
import torch
from transformers import SwinForImageClassification, CvtForImageClassification, ResNetForImageClassification
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
import numpy as np
import pandas as pd
import os
import io
import contextlib

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
dataset = load_dataset("Tsomaros/Chest_Xray_N_Hot3")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/333 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/91 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/91 [00:00<?, ?it/s]

train-00000-of-00091.parquet:   0%|          | 0.00/499M [00:00<?, ?B/s]

...

train-00088-of-00091.parquet:   0%|          | 0.00/506M [00:00<?, ?B/s]

train-00089-of-00091.parquet:   0%|          | 0.00/504M [00:00<?, ?B/s]

train-00090-of-00091.parquet:   0%|          | 0.00/501M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112120 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/87 [00:00<?, ?it/s]

In [ ]:
split = dataset['train'].train_test_split(test_size=0.05)
dataset["train"] = split["train"]
dataset["validation"] = split["test"]

In [ ]:
import matplotlib.pyplot as plt
from torchvision import transforms

class XRayTransform_Train:
    """
    Transforms for pre-processing XRay data across a batch.
    """
    def __init__(self):
        self.transforms = transforms.Compose([
            transforms.Lambda(lambda pil_img: pil_img.convert("RGB")),
            transforms.Resize(feature_extractor.size["height"]),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
        ])

    def __call__(self, example_batch):
        example_batch["pixel_values"] = [self.transforms(pil_img) for pil_img in example_batch["image"]]
        return example_batch

class XRayTransform_Val:
    """
    Transforms for pre-processing XRay data across a batch.
    """
    def __init__(self):
        self.transforms = transforms.Compose([
            transforms.Lambda(lambda pil_img: pil_img.convert("RGB")),
            transforms.Resize(feature_extractor.size["height"]),
            transforms.ToTensor(),
            transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
        ])

    def __call__(self, example_batch):
        example_batch["pixel_values"] = [self.transforms(pil_img) for pil_img in example_batch["image"]]
        return example_batch

In [ ]:
model_name = 'microsoft/swin-base-patch4-window7-224'
model_id = "swin-base-patch4-window7-224"
image_res = (3,224,224)
feature_extractor  = AutoImageProcessor.from_pretrained(model_name)


# Set the training transforms
dataset["train"].set_transform(XRayTransform_Train())
# Set the validation transforms
dataset["validation"].set_transform(XRayTransform_Val())

preprocessor_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
def batch_sampler(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["labels"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
unique_labels = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SwinForImageClassification.from_pretrained(model_name, ignore_mismatched_sizes=True, num_labels=len(unique_labels))

model.to(device)

config.json:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/352M [00:00<?, ?B/s]

Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([15]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([15, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SwinForImageClassification(
  (swin): SwinModel(
    (embeddings): SwinEmbeddings(
      (patch_embeddings): SwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): SwinEncoder(
      (layers): ModuleList(
        (0): SwinStage(
          (blocks): ModuleList(
            (0): SwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): SwinAttention(
                (self): SwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )
                (output): SwinSelfOutput(

In [ ]:
training_args  = TrainingArguments(
    output_dir='./result',
    overwrite_output_dir = True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    dataloader_num_workers=2,
    dataloader_drop_last=True,
    num_train_epochs=10,
    seed=1337,
    logging_steps=50,
    save_steps=1000,
    remove_unused_columns=False,
    warmup_ratio=0.25,
    lr_scheduler_type="cosine",
    learning_rate=2e-4,
    ignore_data_skip=True,
    #push_to_hub=True,
    #metric_for_best_model="roc_auc",
    #load_best_model_at_end=True,
    #eval_strategy="epoch",
    #save_strategy="epoch"
)

In [ ]:
from scipy.special import softmax
import evaluate

metric_auc = evaluate.load("roc_auc", "multilabel")

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)

    pred_scores = softmax(p.predictions.astype('float32'), axis=1)
    auc = metric_auc.compute(prediction_scores=pred_scores, references=p.label_ids, multi_class='ovo')['roc_auc']
    return {"roc_auc": auc}

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
    tokenizer=feature_extractor,
    data_collator=batch_sampler,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

<ipython-input-11-4d78c22f1b4d>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
import transformers

last_checkpoint = None
if os.path.isdir(training_args.output_dir) and not training_args.overwrite_output_dir:
    last_checkpoint = transformers.trainer_utils.get_last_checkpoint(training_args.output_dir)

In [ ]:
# Capture the command line output for plotting loss and learning rate
output = io.StringIO()

with contextlib.redirect_stdout(output):
    trainer.train(resume_from_checkpoint = last_checkpoint)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dimblax99 (dimblax99-patras-events) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,0.729600
100,0.608300
150,0.379000
200,0.240600
250,0.211400
300,0.224700
350,0.215800
400,0.208100
450,0.214800
500,0.205900


In [17]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =       10.0
  eval_loss               =     0.1694
  eval_roc_auc            =      0.844
  eval_runtime            = 0:02:09.28
  eval_samples_per_second =     43.361
  eval_steps_per_second   =      2.715


In [18]:
from ptflops import get_model_complexity_info

macs, params = get_model_complexity_info(model, image_res, as_strings=False, print_per_layer_stat=False, verbose=False)
macs = round(int(macs) / 1000000000, 1)
params = round(int(params) / 1000000, 1)

print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Computational complexity:       15.5    
Number of parameters:           86.8    


In [19]:
import json

filename = '/content/result/eval_results.json'

with open(filename) as fp:
  dictObj = json.load(fp)

dictObj.update({"parameters": params,
                "GMacs": macs,
                "Device": torch.cuda.get_device_name(0),
                "Model Id": model_id})

In [20]:
with open(filename, 'w') as json_file:
    json.dump(dictObj, json_file,
                        indent=4,
                        separators=(',',': '))

In [21]:
model_id = "swin-base-patch4-window7-224"

In [22]:
from google.colab import drive
drive.mount('/content/drive')

%mkdir /content/drive/MyDrive/Medical/Chest_Aug/{model_id}
%cp /content/result/eval_results.json /content/drive/MyDrive/Medical/Chest_Aug/{model_id}

Mounted at /content/drive
